In [1]:
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms

In [3]:
# Simulate arguments using argparse.Namespace
args = argparse.Namespace(
    seed=1,
    seed_data=1,
    unlabeled_weight=1.0,
    batch_size=100,
    count=10,
    balance=True,
    epochs=300,
    labels='/path/to/labels/file'  # specify the correct path for the labels file
)




In [4]:
# Seed setting
torch.manual_seed(args.seed)
np.random.seed(args.seed_data)

In [5]:
# Gaussian Noise Layer
class GaussianNoise(nn.Module):
    def __init__(self, sigma):
        super(GaussianNoise, self).__init__()
        self.sigma = sigma

    def forward(self, x):
        if self.training:  # Only add noise during training
            noise = torch.randn_like(x) * self.sigma
            return x + noise
        return x

In [6]:
# Generator Model
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = nn.Sequential(nn.Linear(100, 500), nn.Softplus())
        self.fc2 = nn.Sequential(nn.Linear(500, 500), nn.Softplus())
        self.fc3 = nn.Linear(500, 28*28)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.sigmoid(self.fc3(x))
        return x

In [7]:
# Discriminator Model
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Sequential(nn.Linear(28*28, 1000), GaussianNoise(0.3))
        self.fc2 = nn.Sequential(nn.Linear(1000, 500), GaussianNoise(0.5))
        self.fc3 = nn.Sequential(nn.Linear(500, 250), GaussianNoise(0.5))
        self.fc4 = nn.Sequential(nn.Linear(250, 250), GaussianNoise(0.5))
        self.fc5 = nn.Linear(250, 10)  # No noise on the last layer

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        return x

In [8]:
# Instantiate models
gen = Generator()
disc = Discriminator()


In [9]:
# Check the models
print(gen)
print(disc)

Generator(
  (fc1): Sequential(
    (0): Linear(in_features=100, out_features=500, bias=True)
    (1): Softplus(beta=1.0, threshold=20.0)
  )
  (fc2): Sequential(
    (0): Linear(in_features=500, out_features=500, bias=True)
    (1): Softplus(beta=1.0, threshold=20.0)
  )
  (fc3): Linear(in_features=500, out_features=784, bias=True)
  (sigmoid): Sigmoid()
)
Discriminator(
  (fc1): Sequential(
    (0): Linear(in_features=784, out_features=1000, bias=True)
    (1): GaussianNoise()
  )
  (fc2): Sequential(
    (0): Linear(in_features=1000, out_features=500, bias=True)
    (1): GaussianNoise()
  )
  (fc3): Sequential(
    (0): Linear(in_features=500, out_features=250, bias=True)
    (1): GaussianNoise()
  )
  (fc4): Sequential(
    (0): Linear(in_features=250, out_features=250, bias=True)
    (1): GaussianNoise()
  )
  (fc5): Linear(in_features=250, out_features=10, bias=True)
)


In [10]:
# Loss and optimizers
criterion = nn.CrossEntropyLoss()
gen_optimizer = optim.Adam(gen.parameters(), lr=0.003)
disc_optimizer = optim.Adam(disc.parameters(), lr=0.003)

In [11]:
# Load MNIST data
transform = transforms.ToTensor()
train_data = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(train_data, batch_size=args.batch_size, shuffle=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 17.5MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 537kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.76MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 8.43MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [12]:
# Training loop
for epoch in range(args.epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape and add noise
        images = images.view(-1, 28*28)

        # Train Discriminator
        disc_optimizer.zero_grad()

        # Label data
        outputs = disc(images)
        loss_lab = criterion(outputs, labels)

        # Unlabeled data (use generator data)
        noise = torch.randn(args.batch_size, 100)
        gen_data = gen(noise).detach()
        outputs_unl = disc(gen_data)
        loss_unl = criterion(outputs_unl, torch.zeros_like(outputs_unl))

        # Calculate discriminator loss and backprop
        disc_loss = loss_lab + args.unlabeled_weight * loss_unl
        disc_loss.backward()
        disc_optimizer.step()

        # Train Generator
        gen_optimizer.zero_grad()
        gen_data = gen(noise)
        gen_loss = criterion(disc(gen_data), torch.ones_like(outputs))
        gen_loss.backward()
        gen_optimizer.step()

    print(f'Epoch [{epoch+1}/{args.epochs}], Loss D: {disc_loss.item():.4f}, Loss G: {gen_loss.item():.4f}')

Epoch [1/300], Loss D: 0.4705, Loss G: 66.0365
Epoch [2/300], Loss D: 0.3871, Loss G: 58.5213
Epoch [3/300], Loss D: 0.5339, Loss G: 42.1819
Epoch [4/300], Loss D: 0.3926, Loss G: 54.6667
Epoch [5/300], Loss D: 0.3469, Loss G: 47.6688
Epoch [6/300], Loss D: 0.3699, Loss G: 34.8319
Epoch [7/300], Loss D: 0.3384, Loss G: 39.5131
Epoch [8/300], Loss D: 0.4349, Loss G: 37.9348
Epoch [9/300], Loss D: 0.2642, Loss G: 70.7694
Epoch [10/300], Loss D: 0.4409, Loss G: 45.7264
Epoch [11/300], Loss D: 0.1207, Loss G: 33.7912
Epoch [12/300], Loss D: 0.2200, Loss G: 52.4639
Epoch [13/300], Loss D: 0.4436, Loss G: 44.0737
Epoch [14/300], Loss D: 0.1933, Loss G: 83.0517
Epoch [15/300], Loss D: 0.3092, Loss G: 74.6155
Epoch [16/300], Loss D: 0.3044, Loss G: 94.3061
Epoch [17/300], Loss D: 0.2052, Loss G: 56.4323
Epoch [18/300], Loss D: 0.2800, Loss G: 73.9703
Epoch [19/300], Loss D: 0.1432, Loss G: 63.5228
Epoch [20/300], Loss D: 0.3599, Loss G: 60.0644
Epoch [21/300], Loss D: 0.2256, Loss G: 78.8735
E